In [1]:
# pip install boto3

In [2]:
import os
import json
import uuid
from datetime import datetime, timezone

import boto3

AWS_REGION = "us-east-2"
TABLE_NAME = "aie_agents_contacts_dev"

dynamodb = boto3.resource("dynamodb", region_name=AWS_REGION)
table = dynamodb.Table(TABLE_NAME)

def now_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat().replace("+00:00", "Z")

def new_contact_id() -> str:
    return f"c_{uuid.uuid4().hex[:12]}"

In [3]:
def pk_contact(contact_id: str) -> str:
    return f"CONTACT#{contact_id}"

def sk_profile() -> str:
    return "PROFILE"

def sk_identity(identity: str) -> str:
    return f"IDENTITY#{identity}"

def pk_identity(identity: str) -> str:
    return f"IDENTITY#{identity}"

def sk_identity_owner() -> str:
    return "CONTACT"

def sk_event(iso_ts: str, event_id: str) -> str:
    return f"EVENT#{iso_ts}#{event_id}"

def sk_summary_current() -> str:
    return "SUMMARY#CURRENT"

def sk_note(iso_ts: str, note_id: str) -> str:
    return f"NOTE#{iso_ts}#{note_id}"

def sk_password() -> str:
    return "PASSWORD"

In [4]:
def create_contact(contact_id: str, display_name: str | None = None, dob: str | None = None, locale: str = "es-MX") -> None:
    ts = now_iso()
    item = {
        "pk": pk_contact(contact_id),
        "sk": sk_profile(),
        "contact_id": contact_id,
        "display_name": display_name or "",
        "dob": dob or "",
        "locale": locale,
        "created_at": ts,
        "updated_at": ts,
    }
    table.put_item(Item=item)

def create_password(contact_id: str, password: str) -> None:
    ts = now_iso()
    item = {
        "pk": pk_contact(contact_id),
        "sk": sk_password(),
        "contact_id": contact_id,
        "password": password,
        "created_at": ts,
        "updated_at": ts,
    }
    table.put_item(Item=item)

def link_identity(contact_id: str, identity: str, channel: str, handle: str, verified: bool = False) -> None:
    ts = now_iso()

    table.put_item(Item={
        "pk": pk_contact(contact_id),
        "sk": sk_identity(identity),
        "contact_id": contact_id,
        "identity": identity,
        "channel": channel,
        "handle": handle,
        "verified": bool(verified),
        "created_at": ts,
        "updated_at": ts,
    })

    table.put_item(Item={
        "pk": pk_identity(identity),
        "sk": sk_identity_owner(),
        "contact_id": contact_id,
        "identity": identity,
        "created_at": ts,
        "updated_at": ts,
    })

def resolve_contact_id(identity: str) -> str | None:
    resp = table.get_item(Key={"pk": pk_identity(identity), "sk": sk_identity_owner()})
    item = resp.get("Item")
    return item.get("contact_id") if item else None

def get_contact_profile(contact_id: str) -> dict | None:
    resp = table.get_item(Key={"pk": pk_contact(contact_id), "sk": sk_profile()})
    return resp.get("Item")

def list_contact_identities(contact_id: str) -> list[dict]:
    resp = table.query(
        KeyConditionExpression="pk = :pk AND begins_with(sk, :prefix)",
        ExpressionAttributeValues={
            ":pk": pk_contact(contact_id),
            ":prefix": "IDENTITY#",
        },
    )
    return resp.get("Items", [])

def add_event(contact_id: str, event_type: str, title: str, details: str, severity: str = "medium", source: str = "agent") -> dict:
    ts = now_iso()
    event_id = f"e_{uuid.uuid4().hex[:10]}"
    item = {
        "pk": pk_contact(contact_id),
        "sk": sk_event(ts, event_id),
        "contact_id": contact_id,
        "event_id": event_id,
        "event_type": event_type,
        "title": title,
        "details": details,
        "severity": severity,
        "source": source,
        "created_at": ts,
        "updated_at": ts,
    }
    table.put_item(Item=item)
    return item

def upsert_summary_current(
    contact_id: str,
    overview: str,
    constraints: list[str] | None = None,
    preferences: list[str] | None = None,
    special_notes: list[str] | None = None,
    sources: list[dict] | None = None,
) -> None:
    ts = now_iso()
    table.put_item(Item={
        "pk": pk_contact(contact_id),
        "sk": sk_summary_current(),
        "contact_id": contact_id,
        "summary": {
            "overview": overview,
            "constraints": constraints or [],
            "preferences": preferences or [],
            "special_notes": special_notes or [],
        },
        "sources": sources or [],
        "last_built_at": ts,
        "created_at": ts,
        "updated_at": ts,
    })

def get_summary_current(contact_id: str) -> dict | None:
    resp = table.get_item(Key={"pk": pk_contact(contact_id), "sk": sk_summary_current()})
    return resp.get("Item")

def get_summary_current(contact_id: str) -> dict | None:
    resp = table.get_item(Key={"pk": pk_contact(contact_id), "sk": sk_summary_current()})
    return resp.get("Item")

def add_note(
    contact_id: str,
    content: str,
    visibility: str = "internal_only",
    source: str = "human",
    tags: list[str] | None = None,
) -> dict:
    ts = now_iso()
    note_id = f"n_{uuid.uuid4().hex[:10]}"
    item = {
        "pk": pk_contact(contact_id),
        "sk": sk_note(ts, note_id),
        "contact_id": contact_id,
        "note_id": note_id,
        "content": content,
        "visibility": visibility,
        "source": source,
        "tags": tags or [],
        "created_at": ts,
        "updated_at": ts,
    }
    table.put_item(Item=item)
    return item

def list_notes(contact_id: str, limit: int | None = None) -> list[dict]:
    resp = table.query(
        KeyConditionExpression="pk = :pk AND begins_with(sk, :prefix)",
        ExpressionAttributeValues={
            ":pk": pk_contact(contact_id),
            ":prefix": "NOTE#",
        },
        Limit=limit,
        ScanIndexForward=False,
    )
    return resp.get("Items", [])

## Cell 4 — Crear un contacto de ejemplo + link WhatsApp

In [5]:
contact_id = new_contact_id()

create_contact(
    contact_id=contact_id,
    display_name="Juan Ignacio Fernández Lrriera",
    dob="1993-02-07",
    locale="es-MX",
)

create_password(
    contact_id=contact_id,
    password="FELJ930207"
)

identity = "whatsapp:525571969848"

link_identity(
    contact_id=contact_id,
    identity=identity,
    channel="whatsapp",
    handle="+525571969848",
    verified=True,
)

print("contact_id:", contact_id)
print("identity:", identity)

contact_id: c_88fe92bfdc65
identity: whatsapp:525571969848


## Cell 5 — Probar reverse lookup identity → contact_id

In [6]:
resolved = resolve_contact_id(identity)
print("resolved contact_id:", resolved)
assert resolved == contact_id

resolved contact_id: c_88fe92bfdc65


## Cell 6 — Leer perfil + listar identidades del contacto

In [7]:
profile = get_contact_profile(contact_id)
identities = list_contact_identities(contact_id)

print(json.dumps(profile, indent=2, ensure_ascii=False))
print(json.dumps(identities, indent=2, ensure_ascii=False))

{
  "updated_at": "2025-12-21T01:45:39Z",
  "contact_id": "c_88fe92bfdc65",
  "created_at": "2025-12-21T01:45:39Z",
  "dob": "1993-02-07",
  "locale": "es-MX",
  "pk": "CONTACT#c_88fe92bfdc65",
  "display_name": "Juan Ignacio Fernández Lrriera",
  "sk": "PROFILE"
}
[
  {
    "updated_at": "2025-12-21T01:45:39Z",
    "contact_id": "c_88fe92bfdc65",
    "handle": "+525571969848",
    "identity": "whatsapp:525571969848",
    "created_at": "2025-12-21T01:45:39Z",
    "pk": "CONTACT#c_88fe92bfdc65",
    "verified": true,
    "channel": "whatsapp",
    "sk": "IDENTITY#whatsapp:525571969848"
  }
]


## Cell 7 — Agregar evento

In [8]:
evt = add_event(
    contact_id=contact_id,
    event_type="SPECIAL_SITUATION",
    title="Responder por mail montos > 0",
    details="Si hay amount > 0, avisar por mail y pedir confirmación.",
    severity="high",
    source="human",
)

print("event sk:", evt["sk"])

event sk: EVENT#2025-12-21T01:45:40Z#e_3633a6a4b4


## Cell 8 — Crear NOTE + Summary

In [9]:
note = add_note(
    contact_id=contact_id,
    content="Prefiere confirmación por mail cuando hay montos > 0. Evitar audios.",
    visibility="internal_only",
    source="human",
    tags=["operativa", "preferencias"],
)

upsert_summary_current(
    contact_id=contact_id,
    overview="Cliente activo. Comunicación clara y directa.",
    constraints=["Si amount > 0, pedir confirmación por mail"],
    preferences=["Mensajes cortos", "WhatsApp como canal principal"],
    special_notes=["En diciembre responde más lento"],
    sources=[
        {"type": "event", "ref_sk": evt["sk"]},
        {"type": "note", "ref_sk": note["sk"]},
    ],
)

print("NOTE:", note["sk"])
print(json.dumps(get_summary_current(contact_id), indent=2, ensure_ascii=False))

NOTE: NOTE#2025-12-21T01:45:40Z#n_c7e70f1a02
{
  "updated_at": "2025-12-21T01:45:40Z",
  "contact_id": "c_88fe92bfdc65",
  "summary": {
    "overview": "Cliente activo. Comunicación clara y directa.",
    "preferences": [
      "Mensajes cortos",
      "WhatsApp como canal principal"
    ],
    "special_notes": [
      "En diciembre responde más lento"
    ],
    "constraints": [
      "Si amount > 0, pedir confirmación por mail"
    ]
  },
  "last_built_at": "2025-12-21T01:45:40Z",
  "created_at": "2025-12-21T01:45:40Z",
  "sources": [
    {
      "type": "event",
      "ref_sk": "EVENT#2025-12-21T01:45:40Z#e_3633a6a4b4"
    },
    {
      "type": "note",
      "ref_sk": "NOTE#2025-12-21T01:45:40Z#n_c7e70f1a02"
    }
  ],
  "pk": "CONTACT#c_88fe92bfdc65",
  "sk": "SUMMARY#CURRENT"
}
